In [8]:
import json
import os
import re

import pdfkit


###此函数用于解析题目和每道题的答案
def jiexi(liebiao):
    new = []
    timu_last = []
    for each in liebiao:
        new.append(re.sub(r'flag=\\"tex\\" ', '', each))
    for each in new:
        timu_last.append(re.sub(r'\\', '', each))
    return timu_last


###此函数用于解析选项
def xuanxiang(liebiao):
    xuanxiang_v2 = []
    xuanxiang_v3 = []
    for each in liebiao:
        a = re.sub('<p>', '', each)
        a = re.sub('</p>', '', a)
        xuanxiang_v2.append(a)
    for each in xuanxiang_v2:
        each = each + '</p>'
        xuanxiang_v3.append(each)
    return xuanxiang_v3


f = open('言语理解与表达错题本2023-07-29，共27页266题.txt', encoding='UTF-8')
line = f.readline().strip()  #读取第一行
txt = []
txt.append(line)
while line:  # 直到读取完文件
    line = f.readline().strip()  # 读取一行文件，包括换行符
    txt.append(line)
f.close()  # 关闭文件

page_text = str(txt)
###获取正确答案
true_answer = re.findall('"correctAnswer": {\', \'"choice": "(.*?)"', page_text, re.S)
###获取解析
solution_list = re.findall(r'"solution": "(.*?)",\', \'"source"', page_text, re.S)
solution_last = jiexi(solution_list)

cailiao = []
timu = []

with open("言语理解与表达错题本2023-07-29，共27页266题.txt", "r", encoding="utf-8") as f:
    content = json.load(f)
###获取单选题题目和复合题的题目
for ieach in content:
    for jeach in ieach['solutions']:
        timu.append(jeach['content'])
        try:
            cailiao.append(jeach['material']['content'])
        except:
            cailiao.append('none')

###获取选项信息
A_option = re.findall('"options": \[\', \'"(.*?)",\', \'".*?",\', \'".*?",\', \'".*?"\', \']', page_text, re.S)
B_option = re.findall('"options": \[\', \'".*?",\', \'"(.*?)",\', \'".*?",\', \'".*?"\', \']', page_text, re.S)
C_option = re.findall('"options": \[\', \'".*?",\', \'".*?",\', \'"(.*?)",\', \'".*?"\', \']', page_text, re.S)
D_option = re.findall('"options": \[\', \'".*?",\', \'".*?",\', \'".*?",\', \'"(.*?)"\', \']', page_text, re.S)

A_option = xuanxiang(A_option)
B_option = xuanxiang(B_option)
C_option = xuanxiang(C_option)
D_option = xuanxiang(D_option)
A_option = jiexi(A_option)
B_option = jiexi(B_option)
C_option = jiexi(C_option)
D_option = jiexi(D_option)

###构造HTML代码
count = 0
all_content = "<!DOCTYPE html>\n<meta charset='utf-8'>\n<html>"
for each in true_answer:
    ###处理复合题
    if cailiao[count] != 'none' and cailiao[count] not in all_content:
        all_content += cailiao[count]
    all_content += str(count + 1)
    all_content += '、'
    all_content += timu[count][3:]
    all_content += 'A、'
    all_content += A_option[count]
    all_content += 'B、'
    all_content += B_option[count]
    all_content += 'C、'
    all_content += C_option[count]
    all_content += 'D、'
    all_content += D_option[count]
    all_content += '<br>'
    count += 1
count = 0
all_content += '<br><br><br><br><br>错题本参考答案<br><br><br><br><br>'
for each in true_answer:
    temp = '第' + str(count + 1) + '题的正确答案为'
    all_content += temp
    if true_answer[count] == '0':
        all_content += 'A'
    elif true_answer[count] == '1':
        all_content += 'B'
    elif true_answer[count] == '2':
        all_content += 'C'
    elif true_answer[count] == '3':
        all_content += 'D'
    all_content += solution_last[count]
    all_content += '<br>'
    count += 1
all_content += '</html>'
name = '言语理解与表达错题本2023-07-29，共27页266题'
path_name = name + '.html'
###保存为HTML文件
with open(path_name, 'w', encoding='utf-8') as fp:
    fp.write(all_content)
confg = pdfkit.configuration(wkhtmltopdf=r'D:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe')
pdfkit.from_url(path_name, name + '.pdf', configuration=confg)  ###把HTML文件转为pdf
print('错题PDF保存成功')
##删除HTML文件
os.remove(path_name)

错题PDF保存成功
